<H1>Готовий до використання алгоритм</H1>

In [20]:
import torch
import torch.utils.data as data
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F
import pandas as pd
import math

from torchmetrics import MeanAbsolutePercentageError
from torchmetrics import MeanSquaredError
from torchmetrics import MeanAbsoluteError

<H2>Завантажуємо натреновану модель</H2>

In [ ]:
#створюємо датасет
class LTV_data(Dataset):
    def __init__(self, df):
        self.df = df
    
    def __getitem__(self, index):
        if columns_float[0] in self.df:
            #якщо маємо модифікований нами фрейм
            feature = self.df.iloc[index][columns_float].values.astype(np.float64)
            target = self.df.iloc[index][columns_target].values.astype(np.float64)
        else:
            #якщо маємо не модифікований фрейм, наприклад, для оцінювання
            feature = []
            target = []
            row = ltv_frame.iloc[index]

            #рахуємо кожну характеристику
            feature.append(row['total_sessions_day7'] - row['total_sessions_day3'])
            feature.append(row['chapters_finished_day7'] - row['chapters_finished_day3'])
            feature.append(row['chapters_opened_day7'] - row['chapters_opened_day3'])
            feature.append(row['chapters_closed_day7'] - row['chapters_closed_day3'])
            feature.append(row['diamonds_received_day7'] - row['diamonds_received_day3'])
            feature.append(row['diamonds_spent_day7'] - row['diamonds_spent_day3'])
            feature.append(row['app_sub_ltv_day3'] - row['app_sub_ltv_day1'])
            feature.append(row['app_iap_ltv_day3'] - row['app_iap_ltv_day1'])
            feature.append(row['ad_ltv_day3'] - row['ad_ltv_day1'])

            target.append(row['target_sub_ltv_day30'] - row['app_sub_ltv_day3'])
            target.append(row['target_iap_ltv_day30'] - row['app_iap_ltv_day3'])
            target.append(row['target_ad_ltv_day30'] - row['ad_ltv_day3'])
            feature = np.array(feature)
        return feature, target

    def __len__(self) -> int:
        return len(self.df)

    @staticmethod
    def create_dataset(df, transform=None):
       dataset = LTV_data(df)
       return dataset

    @staticmethod
    def loader(dataset, batch_size, num_workers=0):
       data_loader = DataLoader(
          dataset,
          batch_size=batch_size,
          shuffle=True,
          num_workers=num_workers,
      )
       return data_loader

In [ ]:
#визначаємо архітектуру моделі
input_size = 9
class LTV_model(nn.Module):
    def __init__(self) -> None:
        super().__init__()
        self.f1 = nn.Sequential(nn.Linear(input_size, 8), nn.Sigmoid(), nn.Linear(8, 4), nn.Sigmoid())
        self.f2 = nn.Linear(4, 3)

    def forward(self, X):
        X = self.f1(X)
        X = self.f2(X)
        return X

In [ ]:
#завантажуємо натреновану модель
model = LTV_model()
checkpoint = torch.load('./model.pt')
model.load_state_dict(checkpoint['model_state_dict'])

In [65]:
#функція для оцінки
def evaluate(model, loader):
    denominator = 10
    print('validation to', len(loader) // denominator, end=':')

    rmse = 0
    mape = 0
    mae = 0

    model.eval()
    
    i = 1
    with torch.no_grad():
        for (features, targets) in loader:
            #для відстежування прогресу
            #if i % denominator == 0:
            #    print(i // denominator, end=';')
            #if i % (denominator * 50) == 0:
            #    print('')
            #i += 1
            
            features = features.float()
            targets = targets.float()

            output = model(features)

            #підраховуємо метрики
            rmse += math.sqrt(MeanSquaredError()(output, targets).item())
            mape += MeanAbsolutePercentageError()(output, targets).item()
            mae += MeanAbsoluteError()(output, targets).item()
        
    rmse = math.sqrt(rmse) / len(loader)
    mape = mape / len(loader)
    mae = mae / len(loader)
    return rmse, mape, mae

In [ ]:
#визначаємо потрібні нам стовпці
#колонки з категоріями
columns_categorical = ['media_source', 'country_code', 'platform']

#колонки з числами
columns_float = ['total_sessions_diff', 'chapters_finished_diff', 'chapters_opened_diff', 'chapters_closed_diff',
'diamonds_received_diff', 'diamonds_spent_diff', 'app_sub_ltv_diff', 'app_iap_ltv_diff', 'ad_ltv_diff']
columns_target = ['target_sub_ltv_diff', 'target_iap_ltv_diff', 'target_ad_ltv_diff']
useful_frame = pd.DataFrame(columns=columns_float + columns_target)

<H2>Завантаження даних для перевірки</H2>

In [31]:
#тут написати ім'я файлу
your_frame = pd.read_csv("data.csv")
your_data = LTV_data.create_dataset(your_frame)
your_loader = LTV_data.loader(your_data, len(your_data))

In [ ]:
#перевіряємо ітеровність
features, targets = next(iter(your_loader))

<H2>Отримання результатів</H2>

In [ ]:
#отримуємо метрики
rmse, mape, mae = evaluate(model, your_loader)
#отримуємо прогноз
prediction = model(next(iter(your_loader)))
print("Test: MSE = %.6f, MAPE = %.4f, MAE = %.6f" % (rmse, mape, mae))